In [1]:
import numpy as np
import math 
import pandas as pd
from sklearn.utils import shuffle #usado apenas para embaralhar o df e esolher dados aleatoriamente
from functools import reduce #transforma lista 2d em lista 1d
import itertools #concatenar

In [8]:
import numpy as np
import math 
import pandas as pd

#classe nó. nó do interno possui mediana que separa planos e folha possui ponto
class Node:
    def __init__(self, left, right, data, depth, dim, classe):
        self.left = left
        self.right = right
        self.data = data
        self.depth = depth
        self.dim = dim
        self.classe = classe
#classe que constroi a árvore       
class KdTree:
    
    def __init__(self):
        self.KdTree = "KdTree"
    
#função que constroi a árvore recursivamente        
    def Build_kdTree(self, df, depth, dim):
        #se estiver na  folha
        if len(df) == 1:
            #guarda info em um no
            leaf = Node(None, None, (df).to_numpy(), depth, dim, df.iloc[-1,-1])
            return leaf
        #se não estiver na folha
        sorted_df = df.sort_values(by = depth%dim)
        cLeft = sorted_df.iloc[:len(sorted_df)//2, :]
        cRight = sorted_df.iloc[len(sorted_df)//2:, :]
        #recursão
        vLeft = self.Build_kdTree(cLeft, depth + 1, dim)
        vRight = self.Build_kdTree(cRight, depth + 1, dim)
        
        #calculos da mediana
        if len(sorted_df)%2 == 0:
            mediana =  ((cRight.iloc[0, depth%dim]) + (cLeft.iloc[-1, depth%dim])) / 2
            #guarda info em um nó
            inner = Node(vLeft, vRight, mediana, depth, dim, None) 
        else:
            mediana = sorted_df.iloc[len(sorted_df)//2, depth%dim]
            inner = Node(vLeft, vRight, mediana, depth, dim, None)
        return inner
  
class x_nn:
    def __init__(self):
        self.x_nn = "x_nn"
    #constroi arvore com treinamento
    def buildTrainedKdTree(self, training_data):
        tree = KdTree()
        dim = len(df.columns)
        tree = tree.Build_kdTree(training_data, 0, dim)  
        return tree
    #retorna lista de fila de prioridade
    def classe(self, test_data, tree, k):
        points = []
        points = test_data.to_numpy()
        queue = []
        classes = []
        ponto_teste = []
        for i in points:
          fila = self.searchKdtree(i, tree, queue, k)
          for j in range(len(fila)):
              classes = list(itertools.chain(classes, fila[j][1]))
          classes = np.array(classes)   
          classes = classes.flatten()
          dicionario = { k : 0 for k in classes}
          
          for k in range(len(classes)):
            dicionario[classes[k]] = dicionario[classes[k]] + 1
          max_key = max(dicionario, key=dicionario.get)
          ponto_teste.append([i, max_key])
        return ponto_teste
      
    #calcula distancia euclediana      
    def euclidean_distance(self, x, y):
        return np.linalg.norm(x-y)
    #insere ponto na lista na ordem certa mantendo o tamanho da fila de prioridade
    def insert(self, lista, ponto):
      pos = 0
      for i in range(len(lista)):
        if(ponto[0] < lista[i][0]):
          queue = [lista[:pos]] + [ponto] + [lista[(pos):]]
          break
        pos =+ 1
      return queue
    #booleano que verifica se ponto pode ser inserido na fila de prioridade
    def compare(self, lista, ponto):
        pos = 0
        for i in lista:
          if(ponto[0] < i[0]):
            return 1
        return 0
    #procura pontos mais proximos com restrição
    def searchKdtree(self, point, tree, queue, k):
      #passo base
      if((tree.left == None) and (tree.right == None)):
        dist = self.euclidean_distance(point, tree.data)
        vaiprafila = [[dist], [tree.classe]]
        #fila ainda nao esta cheia
        if(len(queue) < k):
          queue = queue + [vaiprafila]
          queue = sorted(queue,key=lambda q:q[0])
        #fila cheia
        if(len(queue) == k):
          if(self.compare(queue, vaiprafila) == 1):
            self.insert(queue, vaiprafila)
        #retorna fila
        return queue
      
      #passo recursivo
      #checa restrição(se mediana menor ou maior que coordenada correspondente do ponto)
      if(point[tree.depth % tree.dim] > tree.data):
        near = tree.right
        oposite = tree.left
      else:
        near = tree.left
        oposite = tree.right
      #chama recursão apenas pro lado favoravel
      queue = self.searchKdtree(point, near, queue, k)
      if(len(queue)<k or point[tree.depth % tree.dim] < tree.data):
        queue = self.searchKdtree(point, oposite, queue, k)
      return queue
    #calculo da acuracia    
    def acuracia(self, pontos_proximos):      
        certo = 0
        errado = 0
        for i in range(len(pontos_proximos)):
          if(pontos_proximos[i][0][2] == pontos_proximos[i][1]):
            certo +=1
          else:
            errado += 1 
        acuracia = certo/(certo+errado)
        return acuracia

In [7]:
#leitura 1
df = pd.read_csv('appendicitis.dat', sep=",", header=None)
df = shuffle(df)

#separa data de treino e de teste
training_data = df.iloc[:len(df) * 7 // 10, :]
test_data = df.iloc[len(df) * 7 // 10:, :]
pontos_proximos=[]

#criar um objeto da classe x_nn
x = x_nn()
#chamar função que faz árvore de treinamento
tree = x.buildTrainedKdTree(training_data)
#chamar função que retorna a classe adivinhada de cada ponto teste
pontos_proximos = x.classe(test_data, tree, 5)
#chamar função que calcula acuracia
acuracia = x.acuracia(pontos_proximos)
print("acuracia appendicitis.dat: ")
print(acuracia)

acuracia appendicitis.dat: 
0.03125


In [9]:
#leitura 2
df = pd.read_csv('monk-2.dat', sep=",", header=None)
df = shuffle(df)

training_data = df.iloc[:len(df) * 7 // 10, :]
test_data = df.iloc[len(df) * 7 // 10:, :]
pontos_proximos=[]

x = x_nn()
tree = x.buildTrainedKdTree(training_data)
pontos_proximos = x.classe(test_data, tree, 5)
acuracia = x.acuracia(pontos_proximos)
print("acuracia monk-2.dat: ")
print(acuracia)

acuracia monk-2.dat: 
0.47692307692307695


In [10]:
#leitura 3
df = pd.read_csv('led7digit.dat', sep=",", header=None)
df = shuffle(df)

training_data = df.iloc[:len(df) * 7 // 10, :]
test_data = df.iloc[len(df) * 7 // 10:, :]
pontos_proximos=[]

x = x_nn()
tree = x.buildTrainedKdTree(training_data)
pontos_proximos = x.classe(test_data, tree, 5)
acuracia = x.acuracia(pontos_proximos)
print("led7digit.dat: ")
print(acuracia)

led7digit.dat: 
0.02
